In [1]:
import json
import os
import pandas as pd
pd.set_option('display.max_rows', None)


topo_dir = "/home/jinyu/maro/maro/simulator/scenarios/supply_chain/topologies"
data_dir = "/home/jinyu/maro/maro/simulator/scenarios/supply_chain/topologies/SCI_1.1/data"

num_sample = 500
src_topo_dir = os.path.join(topo_dir, f"SCI_{num_sample}")
src_config = os.path.join(src_topo_dir, "config.yml")

SHOW_ROUTE_INFO = False
if SHOW_ROUTE_INFO:
    route_file = os.path.join(data_dir, "routes.parquet")
    df_route = pd.read_parquet(route_file)

In [2]:
from enum import Enum

class read_stage(Enum):
    to_read_down_facility = 0
    to_read_sku_name = 1
    to_read_up_facility = 2
    to_read_route_name = 3
    to_read_vlt = 4
    to_read_cost = 5
    to_read_down_facility_or_sku_name = 6

In [3]:
def filter_routes(src_cfg, dest_cfg, vendor_dict):
    before_topology = True
    start_settings = False

    down_facility, sku_name, up_facility, route_name, is_first = None, None, None, None, True
    status = read_stage.to_read_down_facility

    with open(src_cfg, 'r') as cfg_in:
        with open(dest_cfg, 'w') as cfg_out:
            for line in cfg_in.readlines():
                if line.startswith("  topology:"):
                    cfg_out.write(line)
                    before_topology = False
                elif line.startswith("settings:"):
                    start_settings = True

                if before_topology or start_settings:
                    cfg_out.write(line)
                else:
                    if status == read_stage.to_read_down_facility_or_sku_name:
                        if line.startswith("    \""):
                            down_facility = line.split("\"")[1]
                            status = read_stage.to_read_sku_name
                            is_first = True
                        elif line.startswith("      \""):
                            sku_name = line.split("\"")[1]
                            status = read_stage.to_read_up_facility
                            is_first = False
                    elif status == read_stage.to_read_down_facility:
                        if line.startswith("    \""):
                            down_facility = line.split("\"")[1]
                            status = read_stage.to_read_sku_name
                    elif status == read_stage.to_read_sku_name:
                        if line.startswith("      \""):
                            sku_name = line.split("\"")[1]
                            status = read_stage.to_read_up_facility
                    elif status == read_stage.to_read_up_facility:
                        if line.startswith("        \""):
                            up_facility = line.split("\"")[1]
                            if up_facility == vendor_dict[down_facility][sku_name][0]:
                                status = read_stage.to_read_route_name
                    elif status == read_stage.to_read_route_name:
                        if line.startswith("          \""):
                            route_name = line.split("\"")[1]
                            if route_name == vendor_dict[down_facility][sku_name][1]:
                                status = read_stage.to_read_vlt
                                # Write down lines
                                if is_first:
                                    cfg_out.write(f"    \"{down_facility}\":\n")
                                cfg_out.write(
                                    f"      \"{sku_name}\":\n"
                                    f"        \"{up_facility}\":\n"
                                    f"          \"{route_name}\":\n"
                                )
                    elif status == read_stage.to_read_vlt:
                        if line.startswith("            vlt"):
                            cfg_out.write(line)
                            status = read_stage.to_read_cost
                    elif status == read_stage.to_read_cost:
                        if line.startswith("            cost"):
                            cfg_out.write(line)
                            cfg_out.write("\n")
                            status = read_stage.to_read_down_facility_or_sku_name

In [4]:
# df_route

In [5]:
# df_route[df_route["EndingLocationId"] == "CA_1"]

In [6]:
vendor_file = os.path.join(src_topo_dir, "shortest_vendor.py")
with open(vendor_file, 'r') as f:
    vendor_dict = json.load(f)

for sku_name in vendor_dict["CA_4"].keys():
    vendor_dict["CA_4"][sku_name] = ["STRG_CA_1", "ROUTE_9"]

vendor_file = os.path.join(src_topo_dir, "shortest_vendor_no_ring.py")
with open(vendor_file, 'w') as f:
    json.dump(vendor_dict, f)

import sys
from pprint import pprint
stdout_fh = sys.stdout
sys.stdout = open(os.path.join(src_topo_dir, "shortest_vendor_no_ring_pprint.py"), 'w')
pprint(vendor_dict)
sys.stdout.close()
sys.stdout = stdout_fh

In [7]:
vendor_file = os.path.join(src_topo_dir, "cheapest_vendor.py")
with open(vendor_file, 'r') as f:
    vendor_dict = json.load(f)

for sku_name in vendor_dict["CA_3"].keys():
    vendor_dict["CA_3"][sku_name] = ["STRG_CA_1", "ROUTE_134"]

for sku_name in vendor_dict["WI_1"].keys():
    vendor_dict["WI_1"][sku_name] = ["STRG_WI_1", "ROUTE_33"]

vendor_file = os.path.join(src_topo_dir, "cheapest_vendor_no_ring.py")
with open(vendor_file, 'w') as f:
    json.dump(vendor_dict, f)

import sys
from pprint import pprint
stdout_fh = sys.stdout
sys.stdout = open(os.path.join(src_topo_dir, "cheapest_vendor_no_ring_pprint.py"), 'w')
pprint(vendor_dict)
sys.stdout.close()
sys.stdout = stdout_fh

In [8]:
for py_name, topo_name in zip(
    ["default_vendor.py", "shortest_vendor_no_ring.py", "cheapest_vendor_no_ring.py"],
    [f"SCI_{num_sample}_default", f"SCI_{num_sample}_shortest_no_ring", f"SCI_{num_sample}_cheapest_no_ring"],
):
    dest_dir = os.path.join(topo_dir, topo_name)
    os.makedirs(dest_dir, exist_ok=True)
    dest_cfg = os.path.join(dest_dir, "config.yml")
    
    vendor_file = os.path.join(src_topo_dir, py_name)
    with open(vendor_file, 'r') as f:
        vendor_dict = json.load(f)
    
    filter_routes(src_config, dest_cfg, vendor_dict)